In [85]:
"""Write a pypsark code to retrieve the top three cities that have the highest number of completed trade orders listed in descending order. Output the city name and the corresponding number of completed trade orders"""

'Write a pypsark code to retrieve the top three cities that have the highest number of completed trade orders listed in descending order. Output the city name and the corresponding number of completed trade orders'

In [90]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
import pyspark.sql.functions as F

In [91]:
spark = SparkSession.builder.appName('Practice').getOrCreate()

In [92]:
schema_trade = StructType([
    StructField('order_id', IntegerType(), True),
    StructField('user_id', IntegerType(), True),
    StructField('price', FloatType(), True),
    StructField('quantity', IntegerType(), True),
    StructField('status', StringType(), True),
    StructField('timestamp', StringType(), True)
])

In [93]:
schema_user = StructType([
    StructField('user_id', IntegerType(), True),
    StructField('city', StringType(), True),
    StructField('email', StringType(), True),
    StructField('signup_date', StringType(), True)
])

In [94]:
data_trade = [
(100101, 111, 9.80, 10, 'Cancelled', '2022-08-17 12:00:00'),
(100102, 111, 10.00, 10, 'Completed', '2022-08-17 12:00:00'),
(100259, 148, 5.10, 35, 'Completed', '2022-08-25 12:00:00'),
(100264, 148, 4.80, 40, 'Completed', '2022-08-26 12:00:00'),
(100305, 300, 10.00, 15, 'Completed', '2022-09-05 12:00:00'),
(100400, 178, 9.90, 15, 'Completed', '2022-09-09 12:00:00'),
(100565, 265, 25.60, 5, 'Completed', '2022-12-19 12:00:00')    
]

In [96]:
data_user = [
(111, 'San Francisco', 'rrok10@gmail.com', '2021-08-03 12:00:00'),
(148, 'Boston', 'sailor9820@gmail.com', '2021-08-20 12:00:00'),
(178, 'San Francisco', 'harrypotterfan182@gmail.com', '2022-01-05 12:00:00'),
(265, 'Denver', 'shadower_@hotmail.com', '2022-02-26 12:00:00'),
(300, 'San Francisco', 'houstoncowboy1122@hotmail.com', '2022-06-3012:00:00')    
]

In [98]:
df_trade = spark.createDataFrame(data_trade, schema_trade)
df_user = spark.createDataFrame(data_user, schema_user)

df_trade.show()
df_user.show()

+--------+-------+-----+--------+---------+-------------------+
|order_id|user_id|price|quantity|   status|          timestamp|
+--------+-------+-----+--------+---------+-------------------+
|  100101|    111|  9.8|      10|Cancelled|2022-08-17 12:00:00|
|  100102|    111| 10.0|      10|Completed|2022-08-17 12:00:00|
|  100259|    148|  5.1|      35|Completed|2022-08-25 12:00:00|
|  100264|    148|  4.8|      40|Completed|2022-08-26 12:00:00|
|  100305|    300| 10.0|      15|Completed|2022-09-05 12:00:00|
|  100400|    178|  9.9|      15|Completed|2022-09-09 12:00:00|
|  100565|    265| 25.6|       5|Completed|2022-12-19 12:00:00|
+--------+-------+-----+--------+---------+-------------------+

+-------+-------------+--------------------+-------------------+
|user_id|         city|               email|        signup_date|
+-------+-------------+--------------------+-------------------+
|    111|San Francisco|    rrok10@gmail.com|2021-08-03 12:00:00|
|    148|       Boston|sailor9820@g

In [110]:
df_agg = df_trade.alias('a') \
    .filter(F.col('a.status') == 'Completed') \
    .join(df_user.alias('b'), F.col('a.user_id') == F.col('b.user_id'), 'left') \
    .groupBy(F.col('b.city')) \
    .agg(
      F.countDistinct(F.col('a.order_id')).alias('order_cnt')  
    ) \
    .select('city') \
    .orderBy(F.col('order_cnt').desc()) \
    .limit(1) \
    .show()

+-------------+
|         city|
+-------------+
|San Francisco|
+-------------+

